# Aug 27, 2025: visualize membership matrices 
on brain, per solution mode

In [1]:
import csv
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp 
import dill as pickle 
from os.path import join as pjoin
from itertools import product
from tqdm import tqdm
from copy import deepcopy
from pathlib import Path
import subprocess
from scipy import sparse, stats
from multiprocessing import Pool
import glob
import random

# import arviz as az

import ants
from nipype.interfaces import afni

from itertools import product, combinations, chain
import multiprocessing as mp
from functools import partial

# networks
import graph_tool.all as gt

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import rainbow

plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr  # CITE ITS PAPER IN YOUR MANUSCRIPT
import colorcet as cc

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

def set_seed(args):
    gt.seed_rng(args.SEED)
    np.random.seed(args.SEED)

set_seed(args)

In [3]:
args.source ='allen' #'spatial' #'allen'
args.space = 'ccfv2' #'yale' #'ccfv2'
args.brain_div = 'whl' #'whl' #'whl'
args.num_rois = 172 #162 #172
args.resolution = 200 #200 #200

PARC_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.resolution}'
)
PARC_DESC

'source-allen_space-ccfv2_braindiv-whl_nrois-172_res-200'

In [4]:
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson'
args.THRESHOLD = f'signed'
args.EDGE_DEF = f'binary'
args.EDGE_DENSITY = 20
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'grp'

BASE_path = f'{os.environ["HOME"]}/new_mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
ROI_path = (
    f'{BASE_path}/roi-results-v3'
    f'/{PARC_DESC}'
)
TS_path = f'{ROI_path}/roi_timeseries'
ROI_RESULTS_path = (
    f'{ROI_path}'
    f'/graph-{args.GRAPH_DEF}/method-{args.GRAPH_METHOD}'
    f'/threshold-{args.THRESHOLD}/edge-{args.EDGE_DEF}/density-{args.EDGE_DENSITY}'
    f'/layer-{args.LAYER_DEF}/unit-{args.DATA_UNIT}'
)
GRAPH_path = f'{ROI_RESULTS_path}/graphs'
os.system(f'mkdir -p {GRAPH_path}')
SBM_path = f'{ROI_RESULTS_path}/model-fits'
os.system(f'mkdir -p {SBM_path}')
DIAG_path = f'{ROI_RESULTS_path}/diagnostics'
os.system(f'mkdir -p {DIAG_path}')
ESTIM_path = f'{ROI_RESULTS_path}/estimates'
os.system(f'mkdir -p {ESTIM_path}/individual')
os.system(f'mkdir -p {ESTIM_path}/group')

0

In [5]:
parcels_img = ants.image_read(f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz')

try:
    roi_labels = pd.read_csv(f'{PARCELS_path}/{PARC_DESC}_desc-names.csv')['roi'].to_list()
except:
    roi_labels = np.arange(1, args.num_rois+1)

In [6]:
# parcels_hr_img = ants.image_read(f'{PARCELS_path}/{PARC_DESC_HR}_desc-parcels.nii.gz')

# roi_labels = pd.read_csv(f'{PARCELS_path}/{PARC_DESC_HR}_desc-names.csv')['roi'].to_list()

In [7]:
parcels_img

ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (57, 40, 66)
	 Spacing    : (0.2, 0.2, 0.2)
	 Origin     : (5.78, 7.83, -6.93)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

In [8]:
def concatenate(in_files, out_file):
    try:
        os.remove(out_file)
    except:
        pass

    tcat = afni.TCat()
    tcat.inputs.in_files = in_files
    tcat.inputs.out_file = out_file
    tcat.inputs.rlt = ''
    tcat.cmdline 
    tcat.run()

    for file in in_files:
        try:
            os.remove(file)
        except:
            pass
    return None

def marginal_to_nifti(args, parcels_img, X, mode_id, folder, level=-1):
    parcels_vol = parcels_img.numpy()
    parcels_vol = np.round(parcels_vol, decimals=0)

    os.system(f'mkdir -p {folder}')
    args.num_rois, args.num_comms = X.shape

    in_files = []
    for idx_comm in range(args.num_comms):
        x = X[:, idx_comm]
        x_img = np.zeros_like(parcels_vol)
        for idx, roi in enumerate(roi_labels):
            x_img += (parcels_vol == roi) * (x[idx])
        
        file = f'{folder}/mode-{mode_id}_comm-{idx_comm}.nii.gz'
        parcels_img.new_image_like(x_img).to_filename(file)
        in_files.append(file)

    if level == -1:
        out_file = f'{folder}/mode-{mode_id:02d}.nii.gz'
    else:
        out_file = f'{folder}/mode-{mode_id:02d}_level-{level}.nii.gz'
    concatenate(in_files, out_file)
    return None

In [9]:
args.dc, args.sbm = False, 'h'

args.nested = True if args.sbm in ['h'] else False

args.force_niter = 40000
args.num_draws = int((1/2) * args.force_niter)

def sbm_name(args):
    dc = f'dc' if args.dc else f'nd'
    dc = f'' if args.sbm in ['m', 'a'] else dc
    file = f'sbm-{dc}-{args.sbm}'
    return file

SBM = sbm_name(args)
SBM

'sbm-nd-h'

In [10]:
def get_membership_matrix(num_rois, df, col='pi'):
    pis = [np.zeros((num_rois, 1)) if np.isnan(pi).all() else pi for pi in df[col]]

    num_modes = len(df)
    num_comms = np.max([pi.shape[-1] for pi in pis])
    num_rois = num_rois
    M = np.zeros((num_rois, num_modes, num_comms)) # membership profile matrix

    for idx_mode, pi in enumerate(pis):
        M[:, idx_mode, :pi.shape[-1]] = pi
    
    return M

In [11]:
marginals_files = sorted(glob.glob(f'{ESTIM_path}/individual/sub-*/partition-modes-group-aligned/{SBM}/desc-mem-mats.pkl', recursive=True))
marginals_df = []
for sbm_file in marginals_files:
    with open(f'{sbm_file}', 'rb') as f:
        row = pickle.load(f)
    marginals_df += [row]
marginals_df = pd.concat(marginals_df).reset_index(drop=True)
mode_ids = list(chain.from_iterable([list(range(count)) for count in marginals_df['sub'].value_counts().sort_index().to_list()]))
marginals_df['mode_id'] = mode_ids
marginals_df

,sub,sbm,pi_0_aligned,pi_1_aligned,pi_2_aligned,pi_3_aligned,pi_4_aligned,pi_5_aligned,pi_6_aligned,pi_7_aligned,pi_8_aligned,omega,mode_id
0,SLC01,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.9496124031007752, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.22194347867143974, 0.6857337728673586, 0.0...","[[0.9264886814453873, 0.003290456272533365, 0....","[[0.9869804103806058, 0.0003403920281931067, 0...","[[0.9990733239571837, 0.0009266760428164416], ...","[[1.0000000000000002], [1.0000000000000002], [...","[[1.0000000000000002], [1.0000000000000002], [...","[[1.0000000000000002], [1.0000000000000002], [...",0.257972,0
1,SLC01,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.9507042253521127, 0.0, 0.03521126760563381...","[[0.2644891195868923, 0.6064414931027854, 0.12...","[[0.9325828479615158, 0.0, 0.04115582101567207...","[[0.984083616669765, 0.0, 0.005627428076316858...","[[0.9929797130142686, 0.0018758093587722862, 0...","[[0.9999999999999999], [1.0], [0.9999999999999...","[[0.9999999999999999], [1.0], [0.9999999999999...","[[0.9999999999999999], [1.0], [0.9999999999999...",0.141756,1
2,SLC01,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.9841269841269842, 0.0, 0.00793650793650793...","[[0.20363102505959652, 0.7645163478496813, 0.0...","[[0.9720796381980593, 0.0024664836694911884, 0...","[[0.9945244451431795, 0.0012332418347455942, 0...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...",0.126375,2
3,SLC01,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.11570247933884298, 0.8099173553719008, 0.0...","[[0.920172529239705, 0.004843284006598568, 0.0...","[[0.9833898982412151, 0.007601647613814924, 0....","[[0.9988425388588039, 0.0011574611411961442], ...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...",0.121384,3
4,SLC01,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.935064935064935, 0.012987012987012988, 0.0...","[[0.45590363157007047, 0.38381372557722526, 0....","[[0.9604266804025352, 0.0, 0.00996918767733052...","[[1.0000000000000002], [1.0], [1.0], [1.0], [0...","[[1.0000000000000002], [1.0], [1.0], [1.0], [0...","[[1.0000000000000002], [1.0], [1.0], [1.0], [0...","[[1.0000000000000002], [1.0], [1.0], [1.0], [0...","[[1.0000000000000002], [1.0], [1.0], [1.0], [0...",0.076945,4
5,SLC01,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.2676056338028169, 0.3380281690140845, 0.39...","[[0.11367826010338059, 0.840491099534684, 0.03...","[[0.9553109376731556, 0.00020075441395560163, ...","[[0.9881620971896524, 0.0, 0.0, 0.011837902810...","[[1.0], [1.0000000000000002], [1.0000000000000...","[[1.0], [1.0000000000000002], [1.0000000000000...","[[1.0], [1.0000000000000002], [1.0000000000000...","[[1.0], [1.0000000000000002], [1.0000000000000...",0.071034,5
6,SLC01,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.8928571428571429, 0.0, 0.01785714285714285...","[[0.3733766233766234, 0.4733324711003283, 0.12...","[[0.9643929709263371, 0.0, 0.00845236555536300...","[[0.9932113341204251, 0.0, 0.00678866587957497...","[[0.9932113341204251, 0.006788665879574971], [...","[[1.0], [0.9999999999999999], [1.0], [0.999999...","[[1.0], [0.9999999999999999], [1.0], [0.999999...","[[1.0], [0.9999999999999999], [1.0], [0.999999...",0.056213,6
7,SLC01,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.9444444444444444, 0.0, 0.05555555555555555...","[[0.18888888888888888, 0.7841269841269841, 0.0...","[[0.9843395691609977, 0.003854875283446712, 0....","[[0.9961451247165533, 0.003854875283446712], [...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [0....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [0....","[[1.0]

In [12]:
cols = [col for col in  list(marginals_df.columns) if 'pi_' in col]
cols

['pi_0_aligned',
 'pi_1_aligned',
 'pi_2_aligned',
 'pi_3_aligned',
 'pi_4_aligned',
 'pi_5_aligned',
 'pi_6_aligned',
 'pi_7_aligned',
 'pi_8_aligned']

In [13]:
# SOFT MARGINALS
soft_marginals_df = []
for sub, group in marginals_df.groupby('sub'):
    omegas = group['omega'].to_list()
    dct = {'sub': [sub], 'sbm': [SBM]}
    for col in cols:
        M = get_membership_matrix(args.num_rois, group, col=col)
        SCs = np.average(M, axis=1, weights=omegas) # soft-comms.
        SCs = np.round(SCs, decimals=3)
        dct[col] = [SCs]
    soft_marginals_df += [pd.DataFrame(dct)]
soft_marginals_df = pd.concat(soft_marginals_df).reset_index(drop=True)
soft_marginals_df

,sub,sbm,pi_0_aligned,pi_1_aligned,pi_2_aligned,pi_3_aligned,pi_4_aligned,pi_5_aligned,pi_6_aligned,pi_7_aligned,pi_8_aligned
0,SLC01,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.873, 0.035, 0.059, 0.0, 0.0, 0.0, 0.0, 0.0...","[[0.228, 0.683, 0.071, 0.0, 0.007, 0.0, 0.011,...","[[0.942, 0.002, 0.033, 0.022], [0.956, 0.001, ...","[[0.988, 0.001, 0.004, 0.007], [0.99, 0.001, 0...","[[0.998, 0.001, 0.001], [0.999, 0.0, 0.001], [...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."


In [14]:
for idx, row in marginals_df.iterrows():
    for col in cols[:]:
        pi = row[col]
        if np.any(np.isnan(pi)): continue
        mode_id = row['mode_id']
        sub = row['sub']
        level = col.split('_')[1]
        if level == 'aligned': level = -1

        # data resolution
        folder = (
            f'{ESTIM_path}/individual/sub-{sub}/partition-modes-group-aligned'
            f'/{SBM}/marginal-visuals/res-{args.resolution}/nii'
        )
        print(sub, mode_id)
        marginal_to_nifti(args, parcels_img, pi, mode_id, folder, level=level)

        # # higher resolution
        # folder = (
        #     f'{ESTIM_path}/individual/sub-{sub}/partition-modes-group-aligned'
        #     f'/{SBM}/marginal-visuals/res-{args.high_res}/nii'
        # )
        # marginal_to_nifti(args, parcels_hr_img, pi, mode_id, folder, level=level)

SLC01 0
250830-09:58:13,323 nipype.interface INFO:
	 stderr 2025-08-30T09:58:13.323616:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
250830-09:58:13,325 nipype.interface INFO:
	 stderr 2025-08-30T09:58:13.325311:++ WARNING: file /home/govindas/.afni.log is now 101329220 (101 million) bytes long!
250830-09:58:13,365 nipype.interface INFO:
	 stderr 2025-08-30T09:58:13.365708:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
250830-09:58:13,367 nipype.interface INFO:
	 stderr 2025-08-30T09:58:13.367207:*+ WARNING: Set TR of output dataset to 1.0 s
250830-09:58:13,492 nipype.interface INFO:
	 stderr 2025-08-30T09:58:13.492261:++ elapsed time = 0.2 s
SLC01 0
250830-09:58:14,287 nipype.interface INFO:
	 stderr 2025-08-30T09:58:14.287799:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
250830-09:58:14,290 nipype.interface INFO:
	 stderr 2025-08-30T09:58:14.290106:++ WARNING: file /home/govindas/.afni.log is now 101337682 (101 million) bytes l

In [15]:
for col in cols[:2]:
    M = get_membership_matrix(args.num_rois, soft_marginals_df, col=col)
    grp_pi = np.median(M, axis=1)
    
    level = col.split('_')[1]
    if level == 'aligned': level = -1

    # data resolution
    folder = (
        f'{ESTIM_path}/group/membership-mats-group-aligned'
        f'/{SBM}/marginal-visuals/res-{args.resolution}/nii'
    ) 
    os.system(f'mkdir -p {folder}')
    marginal_to_nifti(args, parcels_img, grp_pi, 0, folder, level)

    # # higher resolution
    # folder = (
    #     f'{ESTIM_path}/group/membership-mats-group-aligned'
    #     f'/{SBM}/marginal-visuals/res-{args.high_res}/nii'
    # ) 
    # os.system(f'mkdir -p {folder}')
    # marginal_to_nifti(args, parcels_hr_img, grp_pi, 0, folder, level)
    # break


250830-09:59:14,896 nipype.interface INFO:
	 stderr 2025-08-30T09:59:14.896631:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
250830-09:59:14,899 nipype.interface INFO:
	 stderr 2025-08-30T09:59:14.899304:++ WARNING: file /home/govindas/.afni.log is now 101607513 (102 million) bytes long!
250830-09:59:14,915 nipype.interface INFO:
	 stderr 2025-08-30T09:59:14.915820:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
250830-09:59:14,917 nipype.interface INFO:
	 stderr 2025-08-30T09:59:14.917645:*+ WARNING: Set TR of output dataset to 1.0 s
250830-09:59:15,62 nipype.interface INFO:
	 stderr 2025-08-30T09:59:15.062281:++ elapsed time = 0.2 s
250830-09:59:16,232 nipype.interface INFO:
	 stderr 2025-08-30T09:59:16.232581:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
250830-09:59:16,235 nipype.interface INFO:
	 stderr 2025-08-30T09:59:16.235399:++ WARNING: file /home/govindas/.afni.log is now 101618057 (102 million) bytes long!
250830-09:59

In [16]:
col.split('_')

['pi', '1', 'aligned']